In [1]:
import pandas as pd

## 1. Importo los datos


In [2]:
wc_historic = pd.read_csv("01_data_collect/output/historic_data.csv")
missing_data = pd.read_csv("01_data_collect/output/fifa_worldcup_missing.csv")
wc_2022_fixtures = pd.read_csv("01_data_collect/output/wc_2022.csv")

### 1.1 Limpio 2022 Fixtures

In [3]:
wc_2022_fixtures
# Tenemos espacios aunque acá no se vean en los fixtures (home, away)

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [4]:
# Los quito con el metodo strip para strings

wc_2022_fixtures['home'] = wc_2022_fixtures['home'].str.strip()

wc_2022_fixtures['away'] = wc_2022_fixtures['away'].str.strip()

### 1.2 Limpio y uno los años faltantes

In [5]:
# Solo hay datos faltantes para el 90 y sobra una en 1938
print(missing_data.head())
## Datos vacios
print(missing_data[missing_data['home'].isnull()].shape) 
print(missing_data[missing_data['away'].isnull()].shape)
print(missing_data[missing_data['score'].isnull()].shape) # No parece haber datos vacios


             home score             away  year
0          Italy    1–0          Austria  1990
1  United States    1–5   Czechoslovakia  1990
2          Italy    1–0    United States  1990
3        Austria    0–1   Czechoslovakia  1990
4          Italy    2–0   Czechoslovakia  1990
(0, 4)
(0, 4)
(0, 4)


In [6]:
# Concateno y limpio
historical_vf = pd.concat([missing_data, wc_historic], ignore_index=True)  # Sirve para concatenar varios df's

# Dropeo duplicados posibles
historical_vf.drop_duplicates(inplace=True)
historical_vf.sort_values('year', inplace=True)

historical_vf

,home,score,away,year
37,Argentina,1–0,France,1930
52,Uruguay,6–1,Yugoslavia,1930
51,Argentina,6–1,United States,1930
50,Paraguay,1–0,Belgium,1930
49,United States,3–0,Paraguay,1930
...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018
864,Serbia,1–2,Switzerland,2018
865,Serbia,0–2,Brazil,2018
867,Germany,0–1,Mexico,2018


### 1.3 Limpieza de registro historico

In [7]:
# Partido ganado por W (registrado como partido ganado por 3-0)
historical_vf[historical_vf['home'].str.contains('Sweden') & historical_vf['away'].str.contains('Austria')]


,home,score,away,year
73,Sweden,3–0Awarded[a],Austria,1938


In [8]:
# Si vfoy a tener un modelo prediciendo por goles quitaré esta observación
delete_index = historical_vf[historical_vf['home'].str.contains('Sweden') & historical_vf['away'].str.contains('Austria')].index
delete_index


Index([73], dtype='int64')

In [10]:
historical_vf.drop(index=delete_index, inplace=True)

In [11]:
# Rpeito ejercicio con cun¿ba y romania
historical_vf[historical_vf['home'].str.contains('Cuba') & historical_vf['away'].str.contains('Romania') & historical_vf['score'].str.contains('3')]



,home,score,away,year
74,Cuba,3–3 (a.e.t.),Romania,1938


In [13]:
delete_index = historical_vf[historical_vf['home'].str.contains('Cuba') & historical_vf['away'].str.contains('Romania') & historical_vf['score'].str.contains('3')].index
delete_index


Index([74], dtype='int64')

In [ ]:
historical_vf.drop(index=delete_index, inplace=True)

In [151]:
# quitar texto extra 
historical_vf

,home,score,away,year
37,Argentina,1–0,France,1930
52,Uruguay,6–1,Yugoslavia,1930
51,Argentina,6–1,United States,1930
50,Paraguay,1–0,Belgium,1930
49,United States,3–0,Paraguay,1930
...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018
864,Serbia,1–2,Switzerland,2018
865,Serbia,0–2,Brazil,2018
867,Germany,0–1,Mexico,2018


In [152]:
historical_vf['score']= historical_vf['score'].str.replace('[^\d–]', '', regex= True)

In [153]:
# Limpieza espacios como en el anterior

historical_vf['home'] = historical_vf['home'].str.strip()
historical_vf['away'] = historical_vf['away'].str.strip()

In [154]:
historical_vf

,home,score,away,year
37,Argentina,1–0,France,1930
52,Uruguay,6–1,Yugoslavia,1930
51,Argentina,6–1,United States,1930
50,Paraguay,1–0,Belgium,1930
49,United States,3–0,Paraguay,1930
...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018
864,Serbia,1–2,Switzerland,2018
865,Serbia,0–2,Brazil,2018
867,Germany,0–1,Mexico,2018


In [155]:
# Goles de local y visitante en variables separadas

historical_vf[['HomeGoals', 'AwayGoals']]=historical_vf['score'].str.split('–', expand=True)
historical_vf


,home,score,away,year,HomeGoals,AwayGoals
37,Argentina,1–0,France,1930,1,0
52,Uruguay,6–1,Yugoslavia,1930,6,1
51,Argentina,6–1,United States,1930,6,1
50,Paraguay,1–0,Belgium,1930,1,0
49,United States,3–0,Paraguay,1930,3,0
...,...,...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018,2,0
864,Serbia,1–2,Switzerland,2018,1,2
865,Serbia,0–2,Brazil,2018,0,2
867,Germany,0–1,Mexico,2018,0,1


In [156]:
historical_vf.drop('score', axis = 1, inplace= True)

In [157]:
# Poniendo Nombres a las columnas

historical_vf.rename(columns={'home':'HomeTeam', 'away':'AwayTeam', 'year':'Year'}, inplace=True)
historical_vf

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
37,Argentina,France,1930,1,0
52,Uruguay,Yugoslavia,1930,6,1
51,Argentina,United States,1930,6,1
50,Paraguay,Belgium,1930,1,0
49,United States,Paraguay,1930,3,0
...,...,...,...,...,...
863,Brazil,Costa Rica,2018,2,0
864,Serbia,Switzerland,2018,1,2
865,Serbia,Brazil,2018,0,2
867,Germany,Mexico,2018,0,1


In [158]:
# Cambiando tipos de dtaos

print(historical_vf.dtypes)

historical_vf =historical_vf.astype({'HomeGoals':int, 'AwayGoals':int, 'Year':int})

print(historical_vf.dtypes)




HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals    object
AwayGoals    object
dtype: object
HomeTeam     object
AwayTeam     object
Year          int32
HomeGoals     int32
AwayGoals     int32
dtype: object


In [159]:
# Nueva columna de total de goles 
historical_vf['TotalGoal'] = historical_vf['AwayGoals'] + historical_vf['HomeGoals']

historical_vf

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoal
37,Argentina,France,1930,1,0,1
52,Uruguay,Yugoslavia,1930,6,1,7
51,Argentina,United States,1930,6,1,7
50,Paraguay,Belgium,1930,1,0,1
49,United States,Paraguay,1930,3,0,3
...,...,...,...,...,...,...
863,Brazil,Costa Rica,2018,2,0,2
864,Serbia,Switzerland,2018,1,2,3
865,Serbia,Brazil,2018,0,2,2
867,Germany,Mexico,2018,0,1,1


## 2 Exportar datos ya limpios

In [160]:
historical_vf.to_csv('02_prepare_data/output/historic_data.csv', index=False)
wc_2022_fixtures.to_csv('02_prepare_data/output/wc_2022.csv', index= False)